## 파일 확인

In [1]:
import pandas as pd
import glob

files = glob.glob('*.csv')
files

['pre_201901.csv',
 'pre_201902.csv',
 'pre_201903.csv',
 'pre_201904.csv',
 'pre_201905.csv',
 'pre_201906.csv',
 'pre_201907.csv',
 'pre_201908.csv',
 'pre_201909.csv',
 'pre_201910.csv',
 'pre_201911.csv',
 'pre_201912.csv']

## Text Crawl 이전 데이터 전처리용

## 양 끝 공백제거, 경향신문 조선일보 제거, 중복 제거
## 중복, 예외 태그 제거
## 메인 카테고리 문화, IT_과학, 지역 제거
## 로또, 부고, 날씨, 오늘의, 시사일본어 중국어, 한겨레 알림, e글 중심 제거

In [8]:
category_droplist = ['문화', 'IT', '지역']
headline_droplist = ['로또', '복권', '부고', '날씨', '궂긴', '오늘의', '시사일본어', '시사중국어', '일 알림', 'e글중심']

for file in files:
    df = pd.read_csv(file, encoding="utf-8-sig")
    df = df.dropna(axis=0)
    df = df.apply(lambda x: x.str.strip(), axis = 1) # 모든 데이터 양 끝 공백 제거

    df = df[df.press != "경향신문"] # 경향신문 데이터 제거
    df = df[df.press != "조선일보"] # 조선일보 데이터 제거
    df = df.drop_duplicates(["url"]) # 중복 제거
    df = df.reset_index(drop=True)
    
    for index in range(0, len(df)): # 중복, 예외 태그 제거
        if df['headline'][index][-2:] == '중복':
            df['headline'][index] = df['headline'][index][0:len(df['headline'][index])-3].strip()
            continue
        if df['headline'][index][-2:] == '예외':
            df['headline'][index] = None
    
    for index in range(0, len(df)): # 문화, IT, 지역 제거
        if df['category'][index][0:2] in category_droplist:
            df['category'][index] = None
            continue
    
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    
    for index in range(0, len(df)): # 부고, 날씨, 오늘의, 시사일본어 중국어, 한겨레 알림, e글 중심 제거
        if '날씨' in df['category'][index]:
            df['category'][index] = None
            continue
        for drop in headline_droplist:
            if drop in df['headline'][index]:
                df['headline'][index] = None
                break
                
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    
    for index in range(0, len(df)): # len 50미만 컷
        if len(df['text'][index]) < 50:
            df['text'][index] = None
            
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    
    df.to_csv("pre_"+file, encoding="utf-8-sig", index=False # 저장시 파일 이름 주의